In [ ]:
import os
import tensorflow as tf
import numpy as np
import tempfile

# print the tensor flow version
print(tf.__version__)

In [ ]:
from utils import display_digit
from utils import unfold_labels

In [ ]:
# setup tensorboard
tensorboard_file_name = tempfile.gettempdir() + "/TensorFlow/19"
writer = tf.summary.FileWriter(tensorboard_file_name)

In [ ]:
# load data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
batch_size = 100
training_set_size = mnist.train.labels.shape[0]

# define the graph
x = tf.placeholder(tf.float32, [None, 784], name = "X")
W = tf.Variable(tf.zeros([784, 10]), name = "W")
b = tf.Variable(tf.zeros([10]), name = "b")
z = tf.add(tf.matmul(x, W), b, name = "z")
y = tf.nn.softmax(z, name = "y")
y_ = tf.placeholder(tf.float32, [None, 10], name = "y_")

In [ ]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]), name = "cross_entropy")
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
with tf.Session() as sess:
    # write the graph to the tensorboard log
    writer.add_graph(sess.graph)
    
    tf.global_variables_initializer().run()

    for _ in range(training_set_size):
      batch_xs, batch_ys = mnist.train.next_batch(batch_size)
      sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    
    # calculate the accuracy on the training set
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1), name = "correct_prediction")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "accuracy")   
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    
    prediction = tf.argmax(y,1)
    pred = prediction.eval(feed_dict={x: mnist.test.images}, session=sess)


In [ ]:
labels = unfold_labels(mnist.test.labels)
errors = np.where(pred != labels)[0]

random_error_indx = errors[np.random.randint(errors.size, size=1)[0]]
display_digit(mnist.test, random_error_indx, pred[random_error_indx])

In [ ]:
tensorboard_file_name